Find cross section of actors and movies?


In [ ]:
// TODO: use papa parse to load tsv files from imdb
var fs = require('fs');
var papa = require('papaparse');
var sqlite3 = require('better-sqlite3');
var importer = require('../Core');
var createTables = importer.import('create movie database tables');

var principals = '/Users/briancullinan/Downloads/title.principals.tsv';
var titles = '/Users/briancullinan/Downloads/title.basics.tsv';
var names = '/Users/briancullinan/Downloads/name.basics.tsv';
var movies = './movies.sqlite.db';

var db = new sqlite3(movies, {});

function importSQL(file, cb) {
    if(typeof file === 'string') {
        file = fs.createReadStream(file);
    }
    var prev = 0;
    var count = 0;
    return new Promise(resolve => papa.parse(file, {
        worker: true,
        delimiter: '\t',
        fastMode: true,
        skipEmptyLines: true,
        chunk: function(results) {
            count++;
            if(count === 1 || count > prev + 1000) {
                prev = count;
                console.log(`${count} - ${results.data.length} - ${results.data[1]}`);
            }
            
            return results.data.map(r => cb(r));
        },
        complete: resolve
    }));
}

function insertPrincipals(data) {
    return db.prepare(`
INSERT OR IGNORE INTO principals
    (tconst, ordering, nconst, category, job, characters)
VALUES (?,?,?,?,?,?)`).run(data);
}

function insertTitles(data) {
    data = typeof data[0] !== 'object' ? [data] : data;
    return db.prepare(`
INSERT OR IGNORE INTO titles
    (tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres)
VALUES ${data.map(d => '(?,?,?,?,?,?,?,?,?)').join(',')}`).run([].concat.apply([], data));
}

function insertNames(data) {
    data = typeof data[0] !== 'object' ? [data] : data;
    return db.prepare(`
INSERT OR IGNORE INTO name
    (nconst, primaryName, birthYear, deathYear, primaryProfession, knownForTitles)
VALUES ${data.map(d => '(?,?,?,?,?,?)').join(',')}`).run([].concat.apply([], data));
}

$$.async();
createTables(db)
    .then(() => importSQL(names, insertNames))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
    // close the database connection
    .then(() => db.close())



/*
params = [[1,2],[3,4],[5,6],[7,8]]

stmt = db.prepare("INSERT INTO test (num1, num2) VALUES (?1,?2)");
stmt.all([params], function(err, res) {
    ...
});
*/



reading notebook /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[4]
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[0] aka list project files
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[1] aka common ignore paths
reading notebook /Users/briancullinan/jupytangular2/Analytics/budgeting.i

reading notebook /Users/briancullinan/jupytangular2/Utilities/bash.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/child process.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/compress.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/dates.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/diff.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/git commands.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/git.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google calendar data.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google calendar graphs.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google calendar.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/google contacts.ipynb
reading notebook /Users/briancullinan/ju

In [ ]:
var sqlite3 = require('better-sqlite3');
var movies = './movies.sqlite.db';
var db = new sqlite3(movies, {});

function getTitles(t) {
    t = typeof t === 'string' ? [t] : t;
    return db.prepare(`
SELECT *
FROM principals
WHERE tconst IN (${t.map(i => '?').join(',')})`).all(t);
}

function getTitlesByName(n) {
    n = typeof n === 'string' ? [n] : n;
    return db.prepare(`
SELECT *
FROM principals
WHERE nconst IN (${n.map(i => '?').join(',')})`).all(n);
}

function getActorsByTitles(t, n) {
    t = typeof t === 'string' ? [t] : t;
    n = typeof n === 'string' ? [n] : n;
    return db.prepare(`
SELECT *
FROM principals
WHERE nconst IN (${n.map(i => '?').join(',')})
AND tconst IN (${t.map(i => '?').join(',')})`).all(n.concat(t));
}

function getActorTitlesIntersection(n, t) {
    var titles, actors;
    return Promise.resolve()
        .then(() => getTitlesByName(n))
        .then(ts => titles = ts.map(t => t.tconst))
        .then(() => getTitles(t))
        .then(as => actors = as.map(a => a.nconst))
        .then(() => getActorsByTitles(titles, actors))
}

// https://v2.sg.media-imdb.com/suggests/w/werner%20herzog.json
$$.async();
getActorTitlesIntersection('nm0001348', 'tt0082694')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
    // close the database connection
    .then(() => db.close())


In [ ]:
var sqlite3 = require('better-sqlite3');
var movies = './movies.sqlite.db';
var db = new sqlite3(movies, {});

function dropTitles() {
    db.prepare(`DROP INDEX titles_titles`).run();
    db.prepare(`DROP INDEX titles_types`).run();
    db.prepare(`DROP TABLE names`).run();
}

db.prepare(`SELECT COUNT(*) FROM name`).run();


how to use elastic search from node?


In [ ]:
var elasticsearch = require('elasticsearch')
var client = elasticsearch.Client({
  host: 'localhost:9200'
})

client.search({
  index: 'books',
  type: 'book',
  body: {
    query: {
      multi_match: {
        query: 'express js',
        fields: ['title', 'description']
      }
    }
  }
}).then(function (response) {
  var hits = response.hits.hits
}, function (error) {
  console.trace(error.message)
})

create movie database tables?


In [ ]:

function createTables(db) {
    db.prepare(`
CREATE TABLE IF NOT EXISTS principals (
    tconst TEXT NOT NULL,
    ordering INT NOT NULL,
    nconst TEXT NOT NULL,
    category TEXT,
    job TEXT,
    characters TEXT,
    UNIQUE(tconst, ordering),
    PRIMARY KEY (tconst, ordering)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS principles_titles
ON principals (tconst);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS principles_names
ON principals (nconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS titles (
    tconst TEXT NOT NULL,
    titleType TEXT NOT NULL,
    primaryTitle TEXT,
    originalTitle TEXT,
    isAdult INTEGER,
    startYear INTEGER,
    endYear INTEGER,
    runtimeMinutes INTEGER,
    genres TEXT,
    UNIQUE(tconst, titleType),
    PRIMARY KEY (tconst, titleType)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS titles_titles
ON titles (tconst);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS titles_types
ON titles (tconst, titleType);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS akas (
    tconst TEXT NOT NULL,
    ordering INTEGER NOT NULL,
    title TEXT NOT NULL,
    region TEXT NOT NULL,
    language TEXT,
    types TEXT,
    attributes TEXT,
    isOriginalTitle INTEGER,
    UNIQUE(tconst, ordering),
    PRIMARY KEY (tconst, ordering)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS akas_titles
ON akas (tconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS crew (
    tconst TEXT NOT NULL,
    directors TEXT,
    writers TEXT,
    UNIQUE(tconst),
    PRIMARY KEY (tconst)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS crew_titles
ON crew (tconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS episode (
    tconst TEXT NOT NULL,
    parentTconst TEXT NOT NULL,
    seasonNumber INTEGER,
    episodeNumber INTEGER,
    UNIQUE(tconst, parentTconst),
    PRIMARY KEY (tconst, parentTconst)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS episode_titles
ON episode (tconst);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS episode_parents
ON episode (tconst, parentTconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS ratings (
    tconst TEXT NOT NULL,
    averageRating INTEGER,
    numVotes INTEGER,
    UNIQUE(tconst),
    PRIMARY KEY (tconst)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS episode_titles
ON ratings (tconst);`).run()
    
    db.prepare(`
CREATE TABLE IF NOT EXISTS name (
    nconst TEXT NOT NULL,
    primaryName TEXT,
    birthYear INTEGER,
    deathYear INTEGER,
    primaryProfession TEXT,
    knownForTitles TEXT,
    UNIQUE(nconst),
    PRIMARY KEY (nconst)
);`).run()
    
    db.prepare(`
CREATE INDEX IF NOT EXISTS name_names
ON name (nconst);`).run()
    
    return Promise.resolve();
}

module.exports = createTables;
